In [ ]:
pip install pandas

In [73]:
import os
import cv2
import json
import numpy as np
import pandas as pd
from urllib.parse import urlparse
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [136]:
# Load and preprocess the dataset

def parse_url(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    return f"{domain}.jpg"

def load_data():
    # Implement loading and preprocessing your dataset 
   
    f = open('lighthouse_report.json')
    data = json.load(f)['lighthouseReports']
    f.close

    ratings = pd.DataFrame(data)
    
    ratings['filepath'] = ratings['url'].apply(parse_url)
    ratings.drop('id', axis=1, inplace=True)
        
    images = []
    for i in range(0,len(ratings)):
        filepath = ratings['filepath'][i]
        images.append(cv2.imread('images/{0}'.format(filepath),0))
    
    return np.array(images), np.array(ratings['seo'])

In [152]:


images, ratings = load_data()
images = images / 255.0  # normalize images

X_train, X_test, y_train, y_test = train_test_split(images, ratings, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(228, 228, 1),padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu',padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)
"""

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

# Predict rating for a new screenshot
def predict_rating(screenshot):
    screenshot = screenshot / 255.0
    screenshot = np.expand_dims(screenshot, axis=0)  # Add batch dimension
    predicted_rating = model.predict(screenshot)
    return predicted_rating[0][0]

# Example usage
new_screenshot = ...  # Load your new screenshot
predicted_rating = predict_rating(new_screenshot)
print(f'Predicted Rating: {predicted_rating}')"""

d:\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_46 (Conv2D)              │ (None, 228, 228, 32)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_46 (MaxPooling2D) │ (None, 114, 114, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_63 (Dropout)            │ (None, 114, 114, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 114, 114, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_47 (MaxPooling2D) │ (None, 57, 57, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_64 (Dropout)            │ (None, 57, 57, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_23 (Flatten)            │ (None, 207936)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 128)            │    26,615,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_65 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,634,721 (101.60 MB)

 Trainable params: 26,634,721 (101.60 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_45" is incompatible with the layer: expected axis -1 of input shape to have value 207936, but received input with shape (None, 200704)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 224, 224), dtype=float32)
  • training=True
  • mask=None